# 03 Genotype classifier

In this notebook we'll explore LIME and Anchors and figure out if we can learn something from black-box ML models.


**Objectives:**
* use Boruta to find the involved features
* train a GBM classifier that predicts genotype
    * CV
    * optimization? 
* explain using LIME
* explain using Anchors

**Refs:**
* https://kkulma.github.io/2017-11-07-automated_machine_learning_in_cancer_detection/

In [14]:
import pandas as pd
import numpy as np
import os
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

sns.set(style="whitegrid")

---

## Load the data

In [15]:
wd = '/media/tmo/data/work/datasets/02_ST'

logcpm_path = wd + '/ashley_21.03.2018/logcpm_merge_20180212.pickle'
meta_path = wd + '/meta/meta.parquet'

In [16]:
%%time
meta_df = pd.read_parquet(meta_path)

CPU times: user 112 ms, sys: 257 ms, total: 370 ms
Wall time: 146 ms


In [17]:
%%time
logcpm_df = pickle.load(open(logcpm_path, "rb"))

logcpm_df.index.name = 'spot_UID'
logcpm_df.reset_index(inplace=True)
logcpm_df.rename(columns={'sampleID': 'slide_ID'}, inplace=True)

CPU times: user 16.4 s, sys: 7.49 s, total: 23.9 s
Wall time: 23.9 s


In [18]:
st_df = logcpm_df.merge(meta_df, how='inner', on=['spot_UID', 'slide_ID'])

In [19]:
st_df['slide_ID'] = st_df['slide_ID'].astype('category', copy=False)
st_df['GT'] = st_df['GT'].astype('category', copy=False)
st_df['age'] = st_df['age_GT'].astype('category', copy=False)
st_df['age_GT'] = st_df['age_GT'].astype('category', copy=False)

In [20]:
n_genes = 46454
gene_columns = st_df.columns[1:n_genes+1]

In [21]:
expression_df = st_df[gene_columns]

In [22]:
assert expression_df.shape == (10327, 46454)

In [26]:
genotype_df = st_df[['GT']]

---

## Extract *all-relevant* feature set

* use Boruta to reduce the transcriptome feature space to only the genes that perform significantly better than their scrambled counterparts.

**Observations:**
* remarkably, boruta spits out way more relevant features than when regressing on `[AB1_StdDev_Yen]`.

In [32]:
boruta_rf = RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_features='sqrt', max_depth=5)

def train_feature_selector(X_df=expression_df,  # the transcriptome expression vectors
                           y_df=genotype_df,    # the Genotype meta data column
                           estimator=boruta_rf, verbose=2, seed=42):  # boruta parameters
    feature_selector = BorutaPy(estimator=estimator, verbose=verbose, random_state=seed, n_estimators='auto')    
    
    X = X_df.as_matrix()
    y = y_df.values.ravel()
    feature_selector.fit(X, y)
    
    return feature_selector

In [ ]:
GT_feature_selector = train_feature_selector()

In [42]:
GT_features = list(gene_columns[GT_feature_selector.support_])

In [50]:
Gm_hits = list(filter(lambda s: str.startswith(s, 'Gm'), GT_features))
len(Gm_hits)

275

In [51]:
mt_hits = list(filter(lambda s: str.startswith(s, 'mt'), GT_features))
len(mt_hits)

34

---

# Train a GBM classifier

* grid search
* random search
* CV evaluation of a model

**Observations:**
* we can train an almost perfect classifier with GBM
* 5000 trees with default SGBM params and early stopping monitor
* probably overfits, try with less trees

In [57]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, ShuffleSplit

In [83]:
EARLY_STOP_WINDOW_LENGTH = 25

DEFAULT_SGBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 10000,  # can be arbitrarily large
    'max_features': 0.1,
    'subsample': 0.9
}

class EarlyStopMonitor:

    def __init__(self, window_length=EARLY_STOP_WINDOW_LENGTH, threshold=0):        
        self.window_length = window_length
        self.threshold = threshold

    def window_boundaries(self, current_round):        
        lo = max(0, current_round - self.window_length + 1)
        hi = current_round + 1

        return lo, hi

    def __call__(self, current_round, estimator, _):        
        if current_round >= self.window_length - 1:
            lo, hi = self.window_boundaries(current_round)
            
            do_stop = np.mean(estimator.oob_improvement_[lo: hi]) < self.threshold
            
            if do_stop:
                print('stopped after rounds: {}'.format(current_round))
            
            return do_stop
        else:
            return False

In [84]:
X = st_df[GT_features]
y = st_df['GT']

In [62]:
%%time
n_folds = 10
test_size = 0.2
cv = ShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=42)

sgbm = GradientBoostingClassifier(random_state=42, **DEFAULT_SGBM_KWARGS)
scores = cross_val_score(sgbm, X, y, cv=cv, n_jobs=-1, fit_params={'monitor': EarlyStopMonitor()}, groups=st_df[['GT', 'age']])

scores

stopped after rounds: 5673
stopped after rounds: 5640
stopped after rounds: 5807
stopped after rounds: 5825
stopped after rounds: 5875
stopped after rounds: 5941
stopped after rounds: 5827
stopped after rounds: 5946
stopped after rounds: 5958
stopped after rounds: 5976


array([0.99854792, 0.99903195, 0.99854792, 0.99951597, 0.99903195,
       1.        , 0.99951597, 0.99854792, 0.99951597, 0.99903195])

* check out performance with 1000 trees
* guess: still quite good inference

In [71]:
%%time
n_folds = 10
test_size = 0.2
cv = ShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=42)

SGBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 1000,
    'max_features': 0.1,
    'subsample': 0.9
}

sgbm = GradientBoostingClassifier(random_state=42, **SGBM_KWARGS)
scores = cross_val_score(sgbm, X, y, cv=cv, n_jobs=-1, 
                         # fit_params={'monitor': EarlyStopMonitor()},  no need for early stopping
                         groups=st_df[['GT', 'age']])

CPU times: user 3.57 s, sys: 26.4 s, total: 29.9 s
Wall time: 2min 27s


In [72]:
scores

array([0.99709584, 0.99612778, 0.99709584, 0.99854792, 0.99709584,
       0.99903195, 0.99806389, 0.99515973, 0.99709584, 0.99806389])

In [89]:
%%time
n_folds = 10
test_size = 0.2
cv = ShuffleSplit(n_splits=n_folds, test_size=test_size, random_state=42)

SGBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 10000,
    'max_features': 0.1,
    'subsample': 0.9
}

sgbm = GradientBoostingClassifier(random_state=42, **SGBM_KWARGS)
scores = cross_val_score(sgbm, X, y, cv=cv, n_jobs=-1, 
                         fit_params={'monitor': EarlyStopMonitor(threshold=0.001)},
                         groups=st_df[['GT', 'age']])

stopped after rounds: 340
stopped after rounds: 338
stopped after rounds: 340
stopped after rounds: 348
stopped after rounds: 347
stopped after rounds: 340
stopped after rounds: 348
stopped after rounds: 340
stopped after rounds: 348
stopped after rounds: 348
CPU times: user 3.35 s, sys: 27.3 s, total: 30.7 s
Wall time: 1min 14s


In [90]:
scores

array([0.99031946, 0.98983543, 0.98547919, 0.99225557, 0.99225557,
       0.99128751, 0.98983543, 0.98789932, 0.9893514 , 0.9893514 ])

---

# Model explanation with LIME